# Impureza GINI

De manera informal, se puede definir la **impureza GINI** como la probabilidad de que una característica (columna) clasifique incorrectamente una observación, posiblemente por no ser la más conveniente o porque no aporta información para clasificar.

De manera más formal, comparto la siguiente definición obtenida con `ChatGPT`:

La **impureza de Gini** es una medida utilizada en la construcción de árboles de decisión para evaluar la pureza de un nodo, es decir, qué tan mezcladas están las clases en ese nodo. Cuanto más pura sea la distribución de las clases en un nodo (es decir, cuanto más pertenezcan todas las observaciones a una misma clase), menor será la impureza de Gini.

## Fórmula
La impureza de Gini para un nodo \( t \) se calcula como:

$Gini(t) = 1 - \sum_{i=1}^{n} p_i^2$

Donde:
- $ p_i $: Proporción de observaciones pertenecientes a la clase \( i \) en el nodo \( t \).
- $ n $: Número total de clases.

## Interpretación
- Si todas las observaciones en un nodo pertenecen a una sola clase (pura), la impureza de Gini será **0**.
- Si las observaciones están distribuidas uniformemente entre las clases, la impureza de Gini será **máxima**, tendiendo a $\frac{n-1}{n}$.

## Ejemplo
Supongamos que un nodo tiene 100 observaciones, y estas están distribuidas en dos clases:
- Clase A: 80 observaciones $( p_A = 0.8 $)
- Clase B: 20 observaciones $( p_B = 0.2 $)

La impureza de Gini sería:

$Gini = 1 - (0.8^2 + 0.2^2) = 1 - (0.64 + 0.04) = 1 - 0.68 = 0.32$

## Uso en Árboles de Decisión
La impureza de Gini se utiliza para decidir dónde dividir un nodo. Se selecciona el punto de división que minimice la impureza promedio ponderada de los nodos resultantes. Esto ayuda a construir un árbol más eficiente al separar las clases de forma efectiva.


## Ejemplo con Python

In [3]:
import pandas as pd

# Ejemplo de juegos de baloncesto, cada observacion es el rendimiento de u jugador

puntos_partido = pd.Series(["alto", "bajo", "alto", "alto", "alto",
                            "alto", "bajo", "alto", "alto", "bajo"])

minutos_partido = pd.Series(["alto", "alto", "bajo", "bajo", "bajo",
                             "alto", "bajo", "bajo", "bajo", "alto"])

rebotes_partido = pd.Series(["alto", "bajo", "bajo", "alto", "bajo",
                             "alto", "bajo", "alto", "bajo", "alto"])

asistencias_partido = pd.Series(["bajo", "bajo", "bajo", "bajo", "bajo",
                                 "bajo", "bajo", "bajo", "bajo", "bajo"])

# 1: Veterano (carrera de cinco años o más)
clase = pd.Series([1, 0, 0, 1, 0, 1, 0, 1, 0, 1])

datos = pd.DataFrame({"puntos": puntos_partido,
                      "minutos": minutos_partido,
                      "asistencias": asistencias_partido,
                      "rebotes": rebotes_partido,                      
                      "clase": clase})

### Interpretación de Impureza GINI

Aquí abajo lo más importante a observar son los resultados del valor de cada impureza para cada característica potencial que podemos utilizar para clasificar a un jugador como veterano o no. 
Por ejemplo, la característica "rebotes" tiene un impureza del 0.0, la más baja y eso nos está indicando que es una característica adecuada para actuar como clasificador. En cambio, la característica "asistencias" tiene una impureza alta de 0.5, lo que indica que no nos estaría aportando suficiente información a la hora de clasificar a un jugador.

In [2]:
def impureza_gini(caracteristica, clase, datos):
    """str, str, DataFrame -> float"""    
    atributo_clase = datos.groupby([caracteristica, clase])[clase].count()
    atributo = datos.groupby([caracteristica])[clase].count()
    procesados = pd.merge(atributo_clase, atributo, on=[caracteristica], 
                          suffixes=('_individual', '_total')) 
    procesados["combinacion"] = (procesados[clase+"_individual"]/
                                 procesados[clase+"_total"])**2
    gini_combinacion = 1 - procesados.groupby([caracteristica, clase+"_total"])["combinacion"].sum()
    gini_pesado = (gini_combinacion * atributo) / atributo.sum() 
    return gini_pesado.sum()


print("puntos -> impureza: %0.4f" % impureza_gini("puntos", "clase", datos))
print("minutos -> impureza: %0.4f" % impureza_gini("minutos", "clase", datos))
print("asistencias -> impureza: %0.4f" % impureza_gini("asistencias", "clase", datos))
print("rebotes -> impureza: %0.4f" % impureza_gini("rebotes", "clase", datos)) 

datos

puntos -> impureza: 0.4762
minutos -> impureza: 0.4167
asistencias -> impureza: 0.5000
rebotes -> impureza: 0.0000


,puntos,minutos,asistencias,rebotes,clase
0,alto,alto,bajo,alto,1
1,bajo,alto,bajo,bajo,0
2,alto,bajo,bajo,bajo,0
3,alto,bajo,bajo,alto,1
4,alto,bajo,bajo,bajo,0
5,alto,alto,bajo,alto,1
6,bajo,bajo,bajo,bajo,0
7,alto,bajo,bajo,alto,1
8,alto,bajo,bajo,bajo,0
9,bajo,alto,bajo,alto,1


### Impureza del atributo 'puntos'

Aquí se desglosará paso a paso las líneas de la función `impureza_gini` utilizada arriba, para el caso del atributo "puntos".

En esta parte en particular, se realizan un par de agrupamientos y merge para obtener luego las probabilidades de que un jugador sea veterano o no en función del total de puntos 'altos' y puntos 'bajos'.

Ej, si tenemos en total 7 jugadores con puntos 'altos', cuántos de esos jugadores son veteranos y cuántos novatos (en términos de probabilidad).

In [4]:
# Cuántos jugadores por categoría de puntos y por clase (novato o veterano)
puntos_y_clase = datos.groupby(["puntos", "clase"])["clase"].count()

print("\n\nJUGADORES POR PUNTOS Y CLASE\n\n", puntos_y_clase)

# Cuántos jugadores por categoría de puntos 
puntos = datos.groupby(["puntos"])["clase"].count()

print("\n\nJUGADORES POR PUNTOS\n\n", puntos)

# Unir ambas series de datos para procesamiento posterior 
jugadores = pd.merge(
    puntos_y_clase,
    puntos,
    on=["puntos"], 
    suffixes=('_indivual', '_total')) 

print("\n\nUNION\n\n", jugadores)



JUGADORES POR PUNTOS Y CLASE

 puntos  clase
alto    0        3
        1        4
bajo    0        2
        1        1
Name: clase, dtype: int64


JUGADORES POR PUNTOS

 puntos
alto    7
bajo    3
Name: clase, dtype: int64


UNION

         clase_indivual  clase_total
puntos                             
alto                 3            7
alto                 4            7
bajo                 2            3
bajo                 1            3


### Cálculo de la impureza GINI para 'puntos'

Recordando la fórmula vista más arriba de la impureza de GINI:

$Gini(t) = 1 - \sum_{i=1}^{n} p_i^2$

In [7]:
# Probabilidad para cada categoría de puntos con respecto a la clase
jugadores["combinaciones"] = (jugadores["clase_indivual"]/jugadores["clase_total"])**2
print(jugadores)

# Impureza gini para cada combinación
gini_por_combinacion = 1 - jugadores.groupby(["puntos", "clase_total"])["combinaciones"].sum()
print("\n\n",gini_por_combinacion)

# Impureza gini para cada combinación con pesos (ya que no se deben considerar equivalente una impureza 
# aportada por los 7 individuos con puntos altos vs los 3 individuos con puntos bajos
gini_con_peso_por_combinacion = (gini_por_combinacion * puntos) / puntos.sum() 
print("\n\n", gini_con_peso_por_combinacion)
print("\n", gini_con_peso_por_combinacion.sum())

        clase_indivual  clase_total  combinaciones
puntos                                            
alto                 3            7       0.183673
alto                 4            7       0.326531
bajo                 2            3       0.444444
bajo                 1            3       0.111111


 puntos  clase_total
alto    7              0.489796
bajo    3              0.444444
Name: combinaciones, dtype: float64


 puntos  clase_total
alto    7              0.342857
bajo    3              0.133333
dtype: float64

 0.4761904761904763
